In [1]:
# used during development to releoad modules every time there is a change
%load_ext autoreload
%autoreload 2

import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
import numpy as np
import pandas as pd
from scipy.stats import norm
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.optimize import minimize
from numpy.linalg import inv

import statsmodels.api as sm
from course_1.risk_kit import Metrics
import nb.edhec_risk_kit_206 as erk
from backtesting import Backtester, EquallyWeighted, CapWeighted,\
GlobalMiminumVariance, BlackLitterman

import ipywidgets as widgets
import ipywidgets as widgets
from IPython.display import display

import warnings
warnings.filterwarnings('ignore')
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'
pd.options.display.float_format = '{:.6f}'.format

m= Metrics()
blit = BlackLitterman()

In [2]:
ind49_rets = erk.get_ind_returns(weighting="vw", n_inds=49)["2013":]
ind49_mcap = erk.get_ind_market_caps(49, weights=True)["2013":]
inds = ['Hlth', 'Fin', 'Whlsl', 'Rtail', 'Food']
rho_ = ind49_rets[inds].corr()
vols_ = (ind49_rets[inds].std()*np.sqrt(12))    
sigma_prior_ =  (vols_.T).dot(vols_) * rho_

In [3]:
sigma_prior_

,Hlth,Fin,Whlsl,Rtail,Food
Hlth,0.108718,0.057064,0.070764,0.062429,0.040410
Fin,0.057064,0.108718,0.084545,0.064723,0.037292
Whlsl,0.070764,0.084545,0.108718,0.080859,0.058843
Rtail,0.062429,0.064723,0.080859,0.108718,0.062519
Food,0.040410,0.037292,0.058843,0.062519,0.108718


In [4]:
def cw_weights(r, cap_weights):
    w = cap_weights.loc[r.index[1]]
    return w / w.sum()

cw_weights(ind49_rets[inds], ind49_mcap[inds])

Hlth    0.042624
Fin     0.184533
Whlsl   0.096043
Rtail   0.537344
Food    0.139456
Name: 2013-02, dtype: float64

In [5]:
w= ind49_mcap[inds].iloc[1]
w_prior = w / w.sum()
w_prior

Hlth    0.042624
Fin     0.184533
Whlsl   0.096043
Rtail   0.537344
Food    0.139456
Name: 2013-02, dtype: float64

In [20]:
w_prior.idxmax(), w_prior.max()

('Rtail', 0.5373435390087914)

In [6]:
# Assumes that Omega is proportional to the variance of the prior
def proportional_prior(sigma, tau, p):
    """
    Returns the He-Litterman simplified Omega
    Inputs:
    sigma: N x N Covariance Matrix as DataFrame
    tau: a scalar
    p: a K x N DataFrame linking Q and Assets
    returns a P x P DataFrame, a Matrix representing Prior Uncertainties
    """
    helit_omega = p.dot(tau * sigma).dot(p.T)
    # Make a diag matrix from the diag elements of Omega
    return pd.DataFrame(np.diag(np.diag(helit_omega.values)),index=p.index, columns=p.index)

from numpy.linalg import inv

def bl(w_prior, sigma_prior, p, q,
                omega=None,
                delta=2.5, tau=.02):
    """
# Computes the posterior expected returns based on 
# the original black litterman reference model
#
# W.prior must be an N x 1 vector of weights, a Series
# Sigma.prior is an N x N covariance matrix, a DataFrame
# P must be a K x N matrix linking Q and the Assets, a DataFrame
# Q must be an K x 1 vector of views, a Series
# Omega must be a K x K matrix a DataFrame, or None
# if Omega is None, we assume it is
#    proportional to variance of the prior
# delta and tau are scalars
    """
    if omega is None:
        omega = proportional_prior(sigma_prior, tau, p)
    # Force w.prior and Q to be column vectors
    # How many assets do we have?
    N = w_prior.shape[0]
    # And how many views?
    K = q.shape[0]
    # First, reverse-engineer the weights to get pi
    pi = implied_returns(delta, sigma_prior,  w_prior)
    # Adjust (scale) Sigma by the uncertainty scaling factor
    sigma_prior_scaled = tau * sigma_prior  
    # posterior estimate of the mean, use the "Master Formula"
    # we use the versions that do not require
    # Omega to be inverted (see previous section)
    # this is easier to read if we use '@' for matrixmult instead of .dot()
    #     mu_bl = pi + sigma_prior_scaled @ p.T @ inv(p @ sigma_prior_scaled @ p.T + omega) @ (q - p @ pi)
    mu_bl = pi + sigma_prior_scaled.dot(p.T).dot(inv(p.dot(sigma_prior_scaled).dot(p.T) + omega).dot(q - p.dot(pi).values))
    # posterior estimate of uncertainty of mu.bl
#     sigma_bl = sigma_prior + sigma_prior_scaled - sigma_prior_scaled @ p.T @ inv(p @ sigma_prior_scaled @ p.T + omega) @ p @ sigma_prior_scaled
    sigma_bl = sigma_prior + sigma_prior_scaled - sigma_prior_scaled.dot(p.T).dot(inv(p.dot(sigma_prior_scaled).dot(p.T) + omega)).dot(p).dot(sigma_prior_scaled)
    return (mu_bl, sigma_bl)

def implied_returns(delta, sigma, w):
    """
Obtain the implied expected returns by reverse engineering the weights
Inputs:
delta: Risk Aversion Coefficient (scalar)
sigma: Variance-Covariance Matrix (N x N) as DataFrame
    w: Portfolio weights (N x 1) as Series
Returns an N x 1 vector of Returns as Series
    """
    ir = delta * sigma.dot(w).squeeze() # to get a series from a 1-column dataframe
    ir.name = 'Implied Returns'
    return ir

# for convenience and readability, define the inverse of a dataframe
def inverse(d):
    """
    Invert the dataframe by inverting the underlying matrix
    """
    return pd.DataFrame(inv(d.values), index=d.columns, columns=d.index)

def w_msr(sigma, mu, scale=True):
    """
    Optimal (Tangent/Max Sharpe Ratio) Portfolio weights
    by using the Markowitz Optimization Procedure
    Mu is the vector of Excess expected Returns
    Sigma must be an N x N matrix as a DataFrame and Mu a column vector as a Series
    This implements page 188 Equation 5.2.28 of
    "The econometrics of financial markets" Campbell, Lo and Mackinlay.
    """
    w = inverse(sigma).dot(mu)
    if scale:
        w = w/sum(w) # fix: this assumes all w is +ve
    return w

def w_star(delta, sigma, mu):
    return (inverse(sigma).dot(mu))/delta

In [7]:
delta = 2.5
ir = implied_returns(delta=delta, sigma=sigma_prior_, w=w_prior)
ir.idxmax(), ir.max()

('Rtail', 0.22376988664397232)

In [8]:
ir.idxmin(), ir.min()

('Hlth', 0.1528544032954898)

In [9]:
#  subjective view
assets = ['Hlth', 'Fin', 'Whlsl', 'Rtail', 'Food']

# Relative View 1: Hlth will outperform Rtail and Whlsl by 3%
q = pd.Series([0.03])

# start with a single view, all zeros and overwrite the specific view
p = pd.DataFrame([0.]*len(assets), index=assets).T

# find the relative market caps of Rtail and Whlsl to split the
# relative outperformance of Hlth

w_whlsl =  w_prior.loc["Whlsl"]/(w_prior.loc["Whlsl"]+w_prior.loc["Rtail"])
w_rtail =  w_prior.loc["Rtail"]/(w_prior.loc["Whlsl"]+w_prior.loc["Rtail"])

p.iloc[0]['Hlth'] = 1.
p.iloc[0]['Whlsl'] = -w_whlsl
p.iloc[0]['Rtail'] = -w_rtail
(p*100).round(1)

,Hlth,Fin,Whlsl,Rtail,Food
0,100.000000,0.000000,-15.200000,-84.800000,0.000000


In [10]:
delta = 2.5
tau = 0.05
# Find the Black Litterman Expected Returns
bl_mu, bl_sigma = bl(w_prior, sigma_prior_, p, q, tau = tau)
(bl_mu*100).round(1)

Hlth    17.900000
Fin     17.000000
Whlsl   19.300000
Rtail   19.900000
Food    14.500000
dtype: float64

In [21]:
bl_mu.idxmin(), bl_mu.min()

('Food', 0.14224076039345213)

In [11]:
w_bl = w_msr(bl_sigma, bl_mu)
w_bl

Hlth    0.275883
Fin     0.184533
Whlsl   0.060673
Rtail   0.339455
Food    0.139456
dtype: float64

In [12]:
w_bl.idxmax(), w_bl.max()

('Rtail', 0.3394554617563841)

In [13]:
w_bl.idxmin(), w_bl.min()

('Whlsl', 0.06067334016592484)

In [14]:
#  subjective view
assets = ['Hlth', 'Fin', 'Whlsl', 'Rtail', 'Food']

# Relative View 1: Hlth will outperform Rtail and Whlsl by 5%
q = pd.Series([0.05])

# start with a single view, all zeros and overwrite the specific view
p = pd.DataFrame([0.]*len(assets), index=assets).T

# find the relative market caps of Rtail and Whlsl to split the
# relative outperformance of Hlth

w_whlsl =  w_prior.loc["Whlsl"]/(w_prior.loc["Whlsl"]+w_prior.loc["Rtail"])
w_rtail =  w_prior.loc["Rtail"]/(w_prior.loc["Whlsl"]+w_prior.loc["Rtail"])

p.iloc[0]['Hlth'] = 1.
p.iloc[0]['Whlsl'] = -w_whlsl
p.iloc[0]['Rtail'] = -w_rtail
(p*100).round(1)

,Hlth,Fin,Whlsl,Rtail,Food
0,100.000000,0.000000,-15.200000,-84.800000,0.000000


In [15]:
delta = 2.5
tau = 0.05
# Find the Black Litterman Expected Returns
bl_mu, bl_sigma = bl(w_prior, sigma_prior_, p, q, tau = tau)
(bl_mu*100).round(1)

Hlth    18.500000
Fin     16.900000
Whlsl   19.200000
Rtail   19.400000
Food    14.200000
dtype: float64

In [16]:
bl_mu.idxmax(), bl_mu.max()

('Rtail', 0.19393217810996072)

In [17]:
w_bl = w_msr(bl_sigma, bl_mu)
w_bl

Hlth    0.325321
Fin     0.184533
Whlsl   0.053177
Rtail   0.297514
Food    0.139456
dtype: float64

In [18]:
w_bl.idxmax(), w_bl.max()

('Hlth', 0.32532060798438733)